# Chicago Festivals

## Importing from a Google Sheets
## Converting to a DataFrame
## Adding Lat Long Column
## Data Cleaning Including converting addresses into somethign that can be pulled in from google maps api using pygeocoder with a rate limit to not get banned from google maps api with an overload query
## 

In [44]:
import pandas as pd
import numpy as np
from pygeocoder import Geocoder
import time #being used for the rate limit of geocoder

https://chrisalbon.com/python/data_wrangling/geocoding_and_reverse_geocoding/

In [4]:
#TEST

#results = Geocoder.reverse_geocode(df['latitude'][0], df['longitude'][0])
#31.3372728, -109.5609559

results = Geocoder.reverse_geocode(31.3372728, -109.5609559)
results.coordinates

(31.337862, -109.5604109)

In [5]:
print(results)

303 E 3rd St, Douglas, AZ 85607, USA


https://stackoverflow.com/questions/19611729/getting-google-spreadsheet-csv-into-a-pandas-dataframe

In [6]:
#TEST
test = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '0Ak1ecr7i0wotdGJmTURJRnZLYlV3M2daNTRubTdwTXc' +
                   '/export?gid=0&format=csv',
                   # Set first column as rownames in data frame
                   index_col=0,
                   # Parse column values to datetime
                   parse_dates=['Quradate']
                  )
test.head(5)  # Same result as @TomAugspurger

,City,region,Res_Comm,mkt_type,Quradate,National_exp,Alabama_exp,Sales_exp,Inventory_exp,Price_exp,Credit_exp
0,Dothan,South_Central-Montgomery-Auburn-Wiregrass-Dothan,Residential,Rural,2010-01-15,2,2,3,2,3,3
10,Foley,South_Mobile-Baldwin,Residential,Suburban_Urban,2010-01-15,4,4,4,4,4,3
12,Birmingham,North_Central-Birmingham-Tuscaloosa-Anniston,Commercial,Suburban_Urban,2010-01-15,2,2,3,2,2,3
38,Brent,North_Central-Birmingham-Tuscaloosa-Anniston,Residential,Rural,2010-01-15,3,3,3,3,3,2
44,Athens,North_Huntsville-Decatur-Florence,Residential,Suburban_Urban,2010-01-15,4,5,4,4,4,4


Pull in a Google Sheets

In [26]:
festivals = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1GfX5jHq2-_beU07t6EmRN1znuCDy3j1uyBBQmUCoT8I' +
                   '/export?gid=0&format=csv',
                   # Set first column as rownames in data frame
                   index_col=0,
                   # Parse column values to datetime
                   parse_dates=[1]
                  )
#https://docs.google.com/spreadsheets/d/1GfX5jHq2-_beU07t6EmRN1znuCDy3j1uyBBQmUCoT8I/edit#gid=0
festivals.head(20)  # Same result as @TomAugspurger

,Unnamed: 1,Unnamed: 2,Unnamed: 3
Happening TODAY,,,
Fest,Date,Location,Website
Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",http://www.choosechicago.com/event/Puerto-Rica...
Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",http://www.prairiefest.com/
Taste of Randolph,June 15 - 17,"West Loop, Randolph St",http://lineup.tasterandolph.com/
Corgi Cavalcade,"Sat, June 16",350 N Laflin,NaN
Parson’s Block Party,"Sat, June 16","Parson’s Chicken, 2952 W Armitage",NaN
Pride Fest,June 16 - 17,Halsted & Waveland,http://www.chicagoevents.com/event.cfm?eid=142
Gold Coast Art Fest,June 16 - 17,"Grant Park, Butler Field",http://amdurproductions.com/gold-coast-art-fai...
Custer Fair,June 16 - 17,"Evanston, Main & Chicago",NaN


For geocoding, we need to submit a string containing an address or location (such as a city) into the geocode function. However, not all strings are formatted in a way that Google’s geo-API can make sense of them. We can text if an input is valid by using the .geocode().valid_address function.

In [8]:
# Verify that an address is valid (i.e. in Google's system)
Geocoder.geocode("4207 N Washington Ave, Douglas, AZ 85607").valid_address

True

Because the output was True, we now know that this is a valid address and thus can print the latitude and longitude coordinates.

In [9]:
# Print the lat/long
results.coordinates

(31.337862, -109.5604109)

But even more interesting, once the address is processed by the Google geo API, we can parse it and easily separate street numbers, street names, etc.

In [10]:
# Find the lat/long of a certain address
result = Geocoder.geocode("7250 South Tucson Boulevard, Tucson, AZ 85756")

In [11]:
# Print the street number
result.street_number

'7250'

In [12]:
# Print the street name
result.route

'South Tucson Boulevard'

In [20]:
festivals

,Unnamed: 1,Unnamed: 2,Unnamed: 3
Happening TODAY,,,
Fest,Date,Location,Website
Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",http://www.choosechicago.com/event/Puerto-Rica...
Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",http://www.prairiefest.com/
Taste of Randolph,June 15 - 17,"West Loop, Randolph St",http://lineup.tasterandolph.com/
Corgi Cavalcade,"Sat, June 16",350 N Laflin,NaN
Parson’s Block Party,"Sat, June 16","Parson’s Chicken, 2952 W Armitage",NaN
Pride Fest,June 16 - 17,Halsted & Waveland,http://www.chicagoevents.com/event.cfm?eid=142
Gold Coast Art Fest,June 16 - 17,"Grant Park, Butler Field",http://amdurproductions.com/gold-coast-art-fai...
Custer Fair,June 16 - 17,"Evanston, Main & Chicago",NaN


In [27]:
festivals.columns = festivals.iloc[0]
list(festivals.columns.values)

['Date', 'Location', 'Website']

In [46]:
for i in festivals.index: #Pandas has a default 0 index
    numbered_index = 0 # using this since we have a string in the index at the moment, it's the name of the festival
    if numbered_index < 5: #special enforcer to not hit the rate limit until I learn to add columns and pull the lat long
        print (festivals.Location.loc[i])
        true_address = Geocoder.geocode(festivals["Location"][i]).valid_address
        print(true_address)
        #print (festivals.Date.loc[i])
        numbered_index += 1
        time.sleep(5) #eventual enforcer to not hit the geocoder rate limit while getting the lat long

Location


GeocoderError: Error OVER_QUERY_LIMIT
Query: https://maps.google.com/maps/api/geocode/json?address=Location&sensor=false&bounds=&region=&language=&components=

Lets take the first row in our data google sheet and see if the street address under the column location is valid.

In [47]:
# Verify that an address is valid (i.e. in Google's system)
Geocoder.geocode(festivals["Location"][9]).valid_address

GeocoderError: Error OVER_QUERY_LIMIT
Query: https://maps.google.com/maps/api/geocode/json?address=4000+N.+Milwaukee+Ave.+%28at+Irving+Park%2FCicero+to+Belle+Plaine%29&sensor=false&bounds=&region=&language=&components=

Testing that I haven't been fully dinged. Only the Boolean check was rate limit exceeded dinged.
Automation needs an api and properly limiting the rates (see checks added to the looped function)

41.9028805,-87.7035663 humbolt park and division street

In [49]:
#results = Geocoder.reverse_geocode(df['latitude'][0], df['longitude'][0])
results = Geocoder.reverse_geocode(41.9028805,-87.7035663)

In [50]:
print(results)

3002 W Division St, Chicago, IL 60622, USA


In [54]:
idx = 0
festivals.insert(loc=idx, column='latitude', value=[41.9028805])
idx = 0
festivals.insert(loc=idx, column='longitude' value=[-87.7035663])
head.festivals(5)

SyntaxError: invalid syntax (<ipython-input-54-9fc4a4e66f93>, line 4)

In [64]:
#festivals['Name']=''
#festivals['latitude']=''
#festivals['longitude']=''
festivals.head(5)

Fest,Date,Location,Website,latitude,longitude
Happening TODAY,,,,,
Fest,Date,Location,Website,,
Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",http://www.choosechicago.com/event/Puerto-Rica...,,
Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",http://www.prairiefest.com/,,
Taste of Randolph,June 15 - 17,"West Loop, Randolph St",http://lineup.tasterandolph.com/,,
Corgi Cavalcade,"Sat, June 16",350 N Laflin,NaN,,


In [62]:
#del festivals['Name']
festivals.head(5)

Fest,Date,Location,Website
Happening TODAY,,,
Fest,Date,Location,Website
Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",http://www.choosechicago.com/event/Puerto-Rica...
Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",http://www.prairiefest.com/
Taste of Randolph,June 15 - 17,"West Loop, Randolph St",http://lineup.tasterandolph.com/
Corgi Cavalcade,"Sat, June 16",350 N Laflin,NaN


In [65]:
#move the lat and long columns to be next to the address
#You can rearrange columns directly by specifying their order:
#df = df[['a', 'y', 'b', 'x']]
list(festivals.columns.values)

['Date', 'Location', 'Website', 'latitude', 'longitude']

In [68]:
festivals = festivals[['Date', 'Location', 'latitude', 'longitude', 'Website']]
list(festivals.columns.values)

['Date', 'Location', 'latitude', 'longitude', 'Website']

In [69]:
festivals.head(5)

Fest,Date,Location,latitude,longitude,Website
Happening TODAY,,,,,
Fest,Date,Location,,,Website
Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",,,http://www.choosechicago.com/event/Puerto-Rica...
Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/
Taste of Randolph,June 15 - 17,"West Loop, Randolph St",,,http://lineup.tasterandolph.com/
Corgi Cavalcade,"Sat, June 16",350 N Laflin,,,NaN


Drop the 2nd row:

Input

df.reindex(df.index.drop(1))

In [74]:
festivals.reindex(festivals.index.drop('Fest'))

Fest,Date,Location,latitude,longitude,Website
Happening TODAY,,,,,
Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",,,http://www.choosechicago.com/event/Puerto-Rica...
Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/
Taste of Randolph,June 15 - 17,"West Loop, Randolph St",,,http://lineup.tasterandolph.com/
Corgi Cavalcade,"Sat, June 16",350 N Laflin,,,NaN
Parson’s Block Party,"Sat, June 16","Parson’s Chicken, 2952 W Armitage",,,NaN
Pride Fest,June 16 - 17,Halsted & Waveland,,,http://www.chicagoevents.com/event.cfm?eid=142
Gold Coast Art Fest,June 16 - 17,"Grant Park, Butler Field",,,http://amdurproductions.com/gold-coast-art-fai...
Custer Fair,June 16 - 17,"Evanston, Main & Chicago",,,NaN
6 Corners BBQ Fest,Hiatus in 2018,4000 N. Milwaukee Ave. (at Irving Park/Cicero ...,,,http://6cornersbbqfest.com/


In [77]:
#festivals.rename(columns = header)
festivals.index

Index([                                         'Fest',
       'Fiestas Puertorriqueñas & Puerto Rican Parade',
                                 'Oswego PrairieFest ',
                                   'Taste of Randolph',
                                     'Corgi Cavalcade',
                                'Parson’s Block Party',
                                         'Pride Fest ',
                                 'Gold Coast Art Fest',
                                         'Custer Fair',
                                 '6 Corners BBQ Fest ',
       ...
                        'Andersonville Midsommarfest ',
                                 'Chicago Blues Fest ',
                                   'Spring Awakening ',
                             'Chicago Naked Bike Ride',
                                'Global Reddit Meetup',
                                'Rock and Roll Market',
                            'Pilsen Food Truck Social',
                              'Wells 

adding an index row with numbers in order and removing the name from within the index

In [83]:
festivals['Index'] = range(1, len(festivals) + 1)
list(festivals.columns.values)
festivals.head(3)
#del festivals.index.name

Fest,Date,Location,latitude,longitude,Website,Index
Fest,Date,Location,,,Website,1
Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",,,http://www.choosechicago.com/event/Puerto-Rica...,2
Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/,3


interestingly the index still precedes the new Index column I created. 
Need to search how to swap the index with another column

In [97]:
print(list(festivals.columns.values))
# Change the order of the columns
festivals = festivals[['Index', 'Date', 'Location', 'latitude', 'longitude', 'Website']]
festivals.head(3)

['Index', 'Date', 'Location', 'latitude', 'longitude', 'Website', 'NeedsToBeIndex']


Fest,Index,Date,Location,latitude,longitude,Website
Fest,Fest,Date,Location,,,Website
Fiestas Puertorriqueñas & Puerto Rican Parade,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",,,http://www.choosechicago.com/event/Puerto-Rica...
Oswego PrairieFest,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


In [96]:
#This does not work at swapping the index new column with the dataframes index
#festivals['Index'] = festivals.index
#the lower code was needed to fix the pulling of the indexes in each row and overwriting the numbers of the index column
#But...might want to create a new fest row to make sure I don't delete it all
#festivals['Index'] = range(1, len(festivals) + 1)
#festivals['NeedsToBeIndex'] = range(1, len(festivals) + 1)
# not good festivals['Index']  = festivals['tempFest']

KeyError: 'tempFest'

In [102]:
print(list(festivals.columns.values))

['TempFest', 'Date', 'Location', 'latitude', 'longitude', 'Website']


In [108]:
festivals.rename(columns={'TempFest': 'Fest'}, inplace=True)

In [103]:
festivals['Index'] = range(1, len(festivals) + 1)

In [106]:
festivals.set_index('Index', inplace=True)

In [109]:
festivals.head(3)

Fest,Fest,Date,Location,latitude,longitude,Website
Index,,,,,,
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",,,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


Now I can try to add the lat long data in manually (since I am getting banned from google geocoder for now)

In [111]:
# df.at['Row', 'Column'] = 10
festivals.at[2,'latitude'] = 41.9028805
festivals.head(3)

Fest,Fest,Date,Location,latitude,longitude,Website
Index,,,,,,
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",41.9029,,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


In [126]:
# df.at['Row', 'Column'] = 10
festivals.at[2,'longitude'] = -87.7035663
festivals.head(3)
#note the funky name is still present as Index

Fest,Fest,Date,Location,latitude,longitude,Website
Index,,,,,,
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",41.9029,-87.7036,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


In [123]:
pd.set_option('display.max_columns', None)  

In [152]:
print (festivals.loc[2]['latitude'])
print (festivals.loc[2]['longitude'])
print(festivals.index)

festivals.head(3)

#del festivals.Index.name
#Note the display issues it is either numpy or pandas that is doing the truncating of these values
# I have tried to set the disply above in pandas, but it dod nothing 

41.9028805
-87.7035663
Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       164, 165, 166, 167, 168, 169, 170, 171, 172, '2'],
      dtype='object', length=173)


Fest,Fest,Date,Location,latitude,longitude,Website
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",41.9029,-87.7036,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


In [159]:
print(festivals.dtypes)
print(festivals.info())
#get all the things you can run on this object
#dir(festivals.index)

Fest
Fest         object
Date         object
Location     object
latitude     object
longitude    object
Website      object
dtype: object
<class 'pandas.core.frame.DataFrame'>
Index: 173 entries, 1 to 2
Data columns (total 6 columns):
Fest         171 non-null object
Date         172 non-null object
Location     164 non-null object
latitude     173 non-null object
longitude    172 non-null object
Website      102 non-null object
dtypes: object(6)
memory usage: 14.5+ KB
None


In [161]:
festivals.head(3)

Fest,Fest,Date,Location,latitude,longitude,Website
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",41.9029,-87.7036,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


In [171]:
festivals.rename_axis('')
#festivals.index = festivals['IndexT']
festivals.columns.names = ['Index']

In [172]:
#https://stackoverflow.com/questions/19851005/rename-pandas-dataframe-index
festivals.head(3)

Index,Fest,Date,Location,latitude,longitude,Website
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,"Humboldt Park, Division Street",41.9029,-87.7036,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


In [173]:
#Humboldt Park, Division Street converted to Humboldt Park and Division Street
festivals.at[2,'Location'] = "Humboldt Park and Division Street"
festivals.head(3)

Index,Fest,Date,Location,latitude,longitude,Website
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,Humboldt Park and Division Street,41.9029,-87.7036,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",,,http://www.prairiefest.com/


In [189]:
manually_pulled_geo = ["41.6756733,-88.3490662"]
print(manually_pulled_geo)

row_manually_pulled = 3

# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in manually_pulled_geo:
    # Try to,
    try:
        # Split the row by comma, convert to float, and append
        # everything before the comma to lat
        lat.append(float(row.split(',')[0]))
        print(lat)
        # Split the row by comma, convert to float, and append
        # everything after the comma to lon
        lon.append(float(row.split(',')[1]))
        print(lon)
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
festivals.at[row_manually_pulled,'latitude'] = lat[0]
festivals.at[row_manually_pulled, 'longitude'] = lon[0]

['41.6756733,-88.3490662']
[41.6756733]
[-88.3490662]


In [190]:
festivals.head(4)

Index,Fest,Date,Location,latitude,longitude,Website
1,Fest,Date,Location,,,Website
2,Fiestas Puertorriqueñas & Puerto Rican Parade,June 14 - 17,Humboldt Park and Division Street,41.9029,-87.7036,http://www.choosechicago.com/event/Puerto-Rica...
3,Oswego PrairieFest,June 14 - 17,"91 Plank Rd, PrairieFest Park, Oswego",41.6757,-88.3491,http://www.prairiefest.com/
4,Taste of Randolph,June 15 - 17,"West Loop, Randolph St",,,http://lineup.tasterandolph.com/


In [191]:
festivals.to_csv('festivals_dataframe_saved.csv')